# Modelo Arima para Solana

Este notebook contém o desenvolvimento do modelo **ARIMA** para o **Solana**

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Coleta de Dados Históricos (2 anos)
sol = yf.Ticker("SOL-USD")
sol_data = sol.history(period="2y", interval="1h")  # Coleta de dados por hora

In [ ]:
# Coluna 'Close' para previsão
data = sol_data[["Close"]].dropna()

In [ ]:
# Divisão dos Dados (80% treino, 20% teste)
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

In [ ]:
# Construção e Treinamento do Modelo ARIMA
model = ARIMA(train_data, order=(5, 1, 0))
model_fit = model.fit()

In [ ]:
# Fazendo as Previsões
predictions = model_fit.forecast(steps=len(test_data))
predicted_prices = predictions.values

In [ ]:
# Cálculo das Métricas do Modelo
mse = mean_squared_error(test_data, predicted_prices)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_data, predicted_prices)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")

In [ ]:
# Previsão dos próximas 4 dias (96 horas)
future_forecast = model_fit.forecast(steps=96)
future_dates = pd.date_range(start=test_data.index[-1] + pd.Timedelta(hours=1), periods=96, freq='H')

In [ ]:
# Identificação dos Melhores Horários para Comprar e Vender
sell_hour_index = np.argmax(future_forecast)
buy_hour_index = np.argmin(future_forecast)

sell_hour = future_dates[sell_hour_index]
buy_hour = future_dates[buy_hour_index]

print(f"Melhor horário para vender Solana: {sell_hour}")
print(f"Melhor horário para comprar Solana: {buy_hour}")


In [ ]:
# Gráfico mostrando a previsão para as próximas 96 horas comparada aos dados passados
plt.figure(figsize=(14, 7))

# Dados históricos de preços
plt.plot(sol_data.index, sol_data['Close'], label="Dados Históricos", color="black")

# Previsões futuras
plt.plot(future_dates, future_forecast, label="Previsão 96 Horas Futuras", color="blue")

# Melhor horário para vender
plt.scatter(sell_hour, future_forecast[sell_hour_index], color="red", label="Melhor Horário para Vender", marker="v", s=100)

# Melhor horário para comprar
plt.scatter(buy_hour, future_forecast[buy_hour_index], color="green", label="Melhor Horário para Comprar", marker="^", s=100)

plt.title("Previsão de Preço de Solana e Melhores Horários para Compra e Venda")
plt.xlabel("Data e Hora")
plt.ylabel("Preço Previsto (USD)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(test_data.index, test_data, color="black", label="Preços Reais")
plt.plot(test_data.index, predicted_prices, color="blue", label="Previsões")
plt.title("Comparação entre Preços Reais e Previsões")
plt.xlabel("Data e Hora")
plt.ylabel("Preço (USD)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
errors = test_data.values.flatten() - predicted_prices

plt.figure(figsize=(14, 7))
plt.plot(test_data.index, errors, color="red", label="Erros de Previsão")
plt.title("Erros de Previsão (Preço Real - Preço Previsto)")
plt.xlabel("Data e Hora")
plt.ylabel("Erro (USD)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(14, 7))

In [ ]:
# Últimos preços reais de teste
plt.plot(test_data.index[-96:], test_data.values[-96:], label="Últimos Preços Reais", color="black")

# Previsões para as próximas 96 horas
plt.plot(future_dates, future_forecast, label="Previsões Próximas 96 Horas", color="blue")

plt.title("Previsões de Preço para as Próximas 96 Horas")
plt.xlabel("Data e Hora")
plt.ylabel("Preço (USD)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50, color="purple", alpha=0.7)
plt.title("Distribuição dos Erros de Previsão")
plt.xlabel("Erro (USD)")
plt.ylabel("Frequência")
plt.show()

In [ ]:
model_fit.save('model_arima_sol.pkl')